# OpenML dataset: **_airfoil_self_noise_**## **Description****Data Description**:

NASA data set, obtained from a series of aerodynamic and acoustic tests of two and three-dimensional airfoil blade sections conducted in an anechoic wind tunnel.

It comprises different size NACA 0012 airfoils at various wind tunnel speeds and angles of attack. The span of the airfoil and the observer position were the same in all of the experiments.

The task is to predict the (scaled) self noise.
**Attribute Description**:

1. *frequency* - in Hertzs
2. *angle_of_attack* - in degrees
3. *chord_length* - in meters
4. *free_stream_velocity* - in meters per second
5. *displacement_thickness* - in meters
6. *sound_pressure* - in decibels (target feature)

In [ ]:
import openml

dataset = openml.datasets.get_dataset(44957)
X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute, dataset_format='dataframe')

In [ ]:
X.head()

In [ ]:
y.head()